In [1]:
# !pip install yfinance
import yfinance as yf

In [2]:
# !pip install transformers
import transformers

/Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
import numpy as np
# import seaborn as sns
pd.core.common.is_list_like = pd.api.types.is_list_like
import datetime

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import torch
%matplotlib inline

In [4]:
accuracy_map = {}

In [5]:
start = datetime.datetime(2008,8,8)
end = datetime.datetime(2014,7,1)
test_start = datetime.datetime(2014,7,2)
test_end = datetime.datetime(2016,7,1)

def prepare_data(data, days=30):
    X, y, date = [], [], []
    for i in range(len(data) - days):
        X.append(data['Close'].values[i:i+days])
        y.append(data['Close'].values[i+days])
        date.append(np.datetime_as_string(data['Date'].values[i+days], unit='D'))
    return np.array(X), np.array(y),date

def accuracy(truth,predictions):
    correct = 0
    total = 0
    for i in range(1,len(truth)):
        total += 1
        if (truth[i]-truth[i-1])/abs(truth[i]-truth[i-1]) == (predictions[i]-truth[i-1])/abs(predictions[i]-truth[i-1]):
            correct += 1
        else:
            continue
            # print(truth[i-1],truth[i],predictions[i])
    # print(f"accuracy{correct/total}, mse{np.mean((truth - predictions) ** 2)}")
    
    return correct/total,np.mean((truth - predictions) ** 2)

def predict_LR(stock_name,company_name):
    train = yf.download(stock_name, start, end)
    test = yf.download(stock_name, test_start, test_end)
    train.reset_index(inplace=True)
    test.reset_index(inplace=True)
    train["Company"]=company_name
    test["Company"]=company_name

    # Prepare the data
    X_train, y_train, date_train = prepare_data(train)
    X_test, y_test,date_test = prepare_data(test)
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions
    predictions = model.predict(X_test)

    # stock_price_map = {key: value for key, value in zip(date_test, y_train)}
    stock_prediction_map = {key: value for key, value in zip(date_test, predictions)}
    

    # Evaluate the model
    # plt.figure(figsize=(10, 5))
    # plt.plot(y_test, label='True Prices')
    # plt.plot(predictions, label='Predicted Prices')
    # plt.legend()
    # plt.title('Stock Price Prediction')
    # plt.show()


    accuracy_map[company_name] = [accuracy(y_test,predictions)]
    return stock_prediction_map
        

In [6]:
company_map = {'Amazon': 'AMZN', 'Meta': 'META', 'Google': 'GOOGL', 'Microsoft': 'MSFT', 'Apple': 'AAPL', 'IBM': 'IBM', 'NVIDIA': 'NVDA', 'Tesla': 'TSLA', 'Salesforce': 'CRM', 'AMD': 'AMD'}
# predict_LR("META","META")
# Switch keys and values
stock_prediction_map = {}
for key, value in company_map.items():
    stock_prediction_map[key] = predict_LR(value,key)
# stock_prediction_map

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_54302/2734537752.py:19: RuntimeWarning: invalid value encountered in scalar divide
  if (truth[i]-truth[i-1])/abs(truth[i]-truth[i-1]) == (predictions[i]-truth[i-1])/abs(predictions[i]-truth[i-1]):
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_54302/2734537752.py:19: RuntimeWarning: invalid value encountered in scalar divide
  if (truth[i]-truth[i-1])/abs(truth[i]-truth[i-1

In [7]:
accuracy_map

{'Amazon': [(0.5221987315010571, 0.27138410217779574)],
 'Meta': [(0.46511627906976744, 2.9811482053333807)],
 'Google': [(0.5031712473572939, 0.2712143662161038)],
 'Microsoft': [(0.5052854122621564, 0.6500360714208605)],
 'Apple': [(0.4608879492600423, 0.20987337371763637)],
 'IBM': [(0.49471458773784355, 4.022022075505506)],
 'NVIDIA': [(0.46511627906976744, 0.00024436089268023816)],
 'Tesla': [(0.4693446088794926, 0.16787077068440348)],
 'Salesforce': [(0.5243128964059197, 2.003914772152267)],
 'AMD': [(0.4630021141649049, 0.014939979324329859)]}

semantic analysis example

In [8]:
from transformers import BertTokenizer, BertModel, pipeline

import torch

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Example sentence
sentence = "The quick brown fox jumps over the lazy dog."

# Tokenize the input sentence and get token IDs
inputs = tokenizer(sentence, return_tensors='pt')

# Get the outputs from the model
with torch.no_grad():
    outputs = model(**inputs)

# The last hidden state of the [CLS] token (sentence-level representation)
sentence_embedding = outputs.last_hidden_state[:, 0, :]
# sentiment_analysis = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')
sentiment_analysis = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
    return_all_scores=True
)
sentences = [ "I love this product! It's amazing.","Google"]
score_map = {"positive":1,"neutral":0, "negative":-1}
# Analyze the sentiment of each sentence
for sentence in sentences:
    result = sentiment_analysis(sentence)[0]
    print(result)
    print(f"Sentence: {sentence}")


/Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[{'label': 'positive', 'score': 0.9930960536003113}, {'label': 'neutral', 'score': 0.005177223589271307}, {'label': 'negative', 'score': 0.0017267182702198625}]
Sentence: I love this product! It's amazing.
[{'label': 'positive', 'score': 0.5264188647270203}, {'label': 'neutral', 'score': 0.23904329538345337}, {'label': 'negative', 'score': 0.23453785479068756}]
Sentence: Google


cosine similarity example

In [9]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')

def compute_embeddings(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    
    # Compute token embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Mean pooling
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Example texts
text1 = "Microsoft"
text2 = "Musharraf to be impeached."

# Compute embeddings for both texts
embedding1 = compute_embeddings(text1)
embedding2 = compute_embeddings(text2)

# Convert embeddings to numpy arrays
embedding1_np = embedding1.numpy()
embedding2_np = embedding2.numpy()

# Compute cosine similarity
similarity = cosine_similarity(embedding1_np, embedding2_np)
print(similarity)

[[0.00980911]]


In [10]:
def sentiment(title_lst,stock_name):
    res = []
    for title in title_lst: 
        if cosine_similarity(compute_embeddings(title.lower()), compute_embeddings(stock_name.lower())) > 0.2 or stock_name.lower() in title.lower():
            # result = sentiment_analysis(title)[0]
            res.append(sentiment_analysis(title))
        else:
            res.append([])
    return res


In [11]:
#read news data into news_dict
import pandas as pd
csv_file_path = 'Combined_News_DJIA.csv'
df = pd.read_csv(csv_file_path)
df = df.drop(columns=['Label'])
news_dict = df.set_index('Date').T.to_dict('list')

In [12]:
#save customized data for training
columns = ['date', 'company', 'stock price', 'sentiment result', 'prediction']
customized_data = pd.DataFrame(columns=columns)

In [13]:
start = datetime.datetime(2008,8,8)
end = datetime.datetime(2014,7,1)
test_start = datetime.datetime(2014,8,14)
test_end = datetime.datetime(2016,7,1)
# Amazon
Amazon = yf.download("AMZN", test_start, test_end)

# Meta
Meta = yf.download("META", test_start, test_end)

# Google
Google = yf.download("GOOG", test_start, test_end)

# Microsoft
Microsoft = yf.download("MSFT", test_start, test_end)

# Apple
Apple = yf.download("AAPL", test_start, test_end)

# IBM
IBM = yf.download("IBM", test_start, test_end)

# Meta
NVIDIA = yf.download("NVDA", test_start, test_end)

# Tesla
Tesla = yf.download("TSLA", test_start, test_end)

# Salesforce
Salesforce = yf.download("CRM", test_start, test_end)

# AMD
AMD = yf.download("AMD", test_start, test_end)

Amazon.reset_index(inplace=True)
Meta.reset_index(inplace=True)
Google.reset_index(inplace=True)
Microsoft.reset_index(inplace=True)
Apple.reset_index(inplace=True)
IBM.reset_index(inplace=True)
NVIDIA.reset_index(inplace=True)
Tesla.reset_index(inplace=True)
Salesforce.reset_index(inplace=True)
AMD.reset_index(inplace=True)

Amazon["Company"]='Amazon'
Meta["Company"]='Meta'
Microsoft["Company"]='Microsoft'
Apple["Company"]='Apple'
Google["Company"]='Google'
IBM["Company"]='IBM'
NVIDIA["Company"]='NVIDIA'
Tesla["Company"]='Tesla'
Salesforce["Company"]='Salesforce'
AMD["Company"]='AMD'

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [14]:
company_map = {
    # "Amazon": Amazon,
    # "Meta": Meta,
    "Google": Google,
    "Microsoft": Microsoft,
    "Apple": Apple,
    "IBM": IBM,
    "NVIDIA": NVIDIA,
    "Tesla": Tesla,
    "Salesforce": Salesforce,
    "AMD": AMD
}
# Display the dictionary
columns = ['date', 'company', 'stock price', 'sentiment result', 'prediction']
for k,v in company_map.items():
    #initialize df
    customized_data = pd.DataFrame(columns=columns)

    company_name = k
    dataframe = v
    for index, row in dataframe.iterrows():
        print(row)
        print(f"Date: {row['Date']}")
        print(f"Company: {row['Company']}")
        new_row = {
        'date': row['Date'],
        'company': company_name,
        'stock price': row['Close'],
        'sentiment result': sentiment(news_dict[row['Date'].strftime('%Y-%m-%d')],company_name),
        'prediction':stock_prediction_map[company_name][row['Date'].strftime('%Y-%m-%d')]
        }
        print(sentiment(news_dict[row['Date'].strftime('%Y-%m-%d')],company_name))
        customized_data = customized_data._append(new_row, ignore_index=True)
    customized_data.to_csv(f'{company_name}.csv', index=False)  
customized_data

Date         2014-08-14 00:00:00
Open                   28.730122
High                   28.815886
Low                    28.465847
Close                  28.653831
Adj Close              28.653831
Volume                  19709966
Company                   Google
Name: 0, dtype: object
Date: 2014-08-14 00:00:00
Company: Google
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
Date         2014-08-15 00:00:00
Open                    28.81389
High                   28.889683
Low                    28.447895
Close                  28.595491
Adj Close              28.595491
Volume                  30383189
Company                   Google
Name: 1, dtype: object
Date: 2014-08-15 00:00:00
Company: Google


/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_54302/2167295310.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  customized_data = customized_data._append(new_row, ignore_index=True)


[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
Date         2014-08-18 00:00:00
Open                   28.726631
High                   29.145481
Low                    28.721146
Close                  29.028303
Adj Close              29.028303
Volume                  25682318
Company                   Google
Name: 2, dtype: object
Date: 2014-08-18 00:00:00
Company: Google
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [[{'label': 'positive', 'score': 0.5698007941246033}, {'label': 'neutral', 'score': 0.1665957272052765}, {'label': 'negative', 'score': 0.26360347867012024}]], [], [[{'label': 'positive', 'score': 0.7075182795524597}, {'label': 'neutral', 'score': 0.11710415035486221}, {'label': 'negative', 'score': 0.1753775179386139}]], [], [], [], [], []]
Date         2014-08-19 00:00:00
Open                   29.169914
High                   29.286594
Low                     29.12005
Close                  29

In [ ]:

customized_data.to_csv('amazon.csv', index=False)  


In [ ]:
customized_data

,date,company,stock price,sentiment result,prediction
0,2014-08-14,Amazon,16.660500,"[(0, 0.0), (0, 0.0), (3, 0.25634676218032837),...",16.334515
1,2014-08-15,Amazon,16.681499,"[(0, 0.0), (0, 0.0), (0, 0.0), (0, 0.0), (0, 0...",16.707203
2,2014-08-18,Amazon,16.726500,"[(0, 0.0), (0, 0.0), (0, 0.0), (0, 0.0), (0, 0...",16.735311
3,2014-08-19,Amazon,16.756500,"[(0, 0.0), (0, 0.0), (0, 0.0), (0, 0.0), (0, 0...",16.632341
4,2014-08-20,Amazon,16.789000,"[(0, 0.0), (0, 0.0), (0, 0.0), (0, 0.0), (0, 0...",16.752566
...,...,...,...,...,...
943,2016-06-24,Amazon,34.948002,"[(0, 0.0), (0, 0.0), (0, 0.0), (0, 0.0), (0, 0...",36.024235
944,2016-06-27,Amazon,34.568001,"[(0, 0.0), (0, 0.0), (0, 0.0), (0, 0.0), (0, 0...",34.976840
945,2016-06-28,Amazon,35.397499,"[(1, 0.44629529118537903), (0, 0.0), (0, 0.0),...",34.642114
946,2016-06-29,Amazon,35.779999,"[(0, 0.0), (0, 0.0), (0, 0.0), (0, 0.0), (0, 0...",35.384107


Hidden Markov Model

In [ ]:
amazon = pd.read_csv('amazon.csv')
amazon

,date,company,stock price,sentiment result,prediction
0,2014-08-14,Amazon,16.660500,"[[], [], [[{'label': 'positive', 'score': 0.22...",16.334515
1,2014-08-15,Amazon,16.681499,"[[], [], [], [], [], [], [], [], [], [], [], [...",16.707203
2,2014-08-18,Amazon,16.726500,"[[], [], [], [], [], [], [], [], [], [], [], [...",16.735311
3,2014-08-19,Amazon,16.756500,"[[], [], [], [], [], [], [], [], [], [], [], [...",16.632341
4,2014-08-20,Amazon,16.789000,"[[], [], [], [], [], [], [], [[{'label': 'posi...",16.752566
...,...,...,...,...,...
469,2016-06-24,Amazon,34.948002,"[[], [], [], [], [], [], [], [], [], [], [], [...",36.024235
470,2016-06-27,Amazon,34.568001,"[[], [], [], [], [], [], [], [], [], [], [], [...",34.976840
471,2016-06-28,Amazon,35.397499,"[[[{'label': 'positive', 'score': 0.5076484680...",34.642114
472,2016-06-29,Amazon,35.779999,"[[], [], [], [], [], [], [], [], [], [], [], [...",35.384107


In [ ]:
def average(data):
    if len(data) <1:
        return [0,0,0]
    data = np.array(data)
    num_rows = data.shape[0]
    # Calculate weights inversely proportional to the index (higher weight for lower index)
    weights = np.arange(num_rows, 0, -1)
    # Optionally, normalize weights to sum up to 1
    weights = weights / np.sum(weights)
    # Calculate the weighted sum for each column
    weighted_sum = np.sum(data.T * weights, axis=1)
    # Calculate the weighted average for each column
    weighted_average = weighted_sum
    return weighted_average

In [ ]:
window = []
amazon_truth = []
amazon_prediction = []
for i, row in amazon.iterrows():
    if len(window) < 4:
        window.append(row['stock price'])
        continue
    window = window[1:]
    sentiment_lst = []
    for sentiment in eval(row["sentiment result"]):
        if len(sentiment) != 0:
            sentiment = sentiment[0]
            temp = [sentiment[0]["score"],sentiment[1]["score"],sentiment[2]["score"]]
            if sentiment[0]["label"]!="positive":
                print("$$error warning")
            sentiment_lst.append(temp) 

    sentiment_array = average(sentiment_lst)
    sig = np.average(np.abs(np.diff(np.array(window))))
    value = np.array([row["prediction"]-sig,row["prediction"],row["prediction"]+sig])
    p = np.array([0.25,0.5,0.25])
    print(p,sentiment_array)
    p = p*sentiment_array
    print("p",p)
    p = p/p.sum(axis=0)
    index = np.argmax(p)
    print("p",p)
    print("index",index)
    revised_prediction = value[index]
    amazon_prediction.append(revised_prediction)
    amazon_truth.append(row['stock price'])
    window.append(row['stock price'])


[0.25 0.5  0.25] [0.08442143 0.09269048 0.82288808]
p [0.02110536 0.04634524 0.20572202]
p [0.07726015 0.1696555  0.75308435]
index 2
[0.25 0.5  0.25] [0.48014168 0.18690988 0.33294844]
p [0.12003542 0.09345494 0.08323711]
p [0.40453086 0.31495211 0.28051703]
index 0
[0.25 0.5  0.25] [0, 0, 0]
p [0. 0. 0.]
p [nan nan nan]
index 0
[0.25 0.5  0.25] [0.4138993  0.21070628 0.37539443]
p [0.10347483 0.10535314 0.09384861]
p [0.34186599 0.34807167 0.31006235]
index 1
[0.25 0.5  0.25] [0.2375662  0.19468516 0.56774867]
p [0.05939155 0.09734258 0.14193717]
p [0.19885256 0.32591877 0.47522868]
index 2
[0.25 0.5  0.25] [0.48013985 0.12556285 0.39429733]
p [0.12003496 0.06278142 0.09857433]
p [0.42657755 0.22311121 0.35031124]
index 0
[0.25 0.5  0.25] [0.22303512 0.12239641 0.65456842]
p [0.05575878 0.0611982  0.16364211]
p [0.19871333 0.21809837 0.5831883 ]
index 2
[0.25 0.5  0.25] [0, 0, 0]
p [0. 0. 0.]
p [nan nan nan]
index 0
[0.25 0.5  0.25] [0, 0, 0]
p [0. 0. 0.]
p [nan nan nan]
index 0
[0.2

/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_37209/2399605597.py:25: RuntimeWarning: invalid value encountered in divide
  p = p/p.sum(axis=0)


In [ ]:
accuracy(amazon_truth,amazon_prediction)


16.579500198364258 16.701000213623047 16.5684819377262
17.09149932861328 17.159000396728516 16.753355376909898
17.159000396728516 17.000999450683594 17.351826467487285
16.95199966430664 17.118999481201172 16.821307838754315
16.950000762939453 17.297500610351562 16.771902508074774
16.525999069213867 16.559499740600586 16.47545645151391
16.559499740600586 16.194499969482422 16.630787994640826
16.25 16.56599998474121 16.20533226286936
16.181499481201172 16.410499572753906 15.998188196096535
16.096500396728516 16.160499572753906 15.841617382448003
16.090999603271484 16.121999740600586 16.079125987856504
15.873000144958496 15.920499801635742 15.70441061786153
15.848999977111816 16.135000228881836 15.743004509037874
15.322500228881836 15.415499687194824 15.091489304259703
15.415499687194824 15.298500061035156 15.446180687975092
15.182000160217283 15.310500144958496 15.065613912893944
15.648500442504885 15.659000396728516 15.333057656498884
15.659000396728516 14.352999687194824 15.70696879133

0.4989339019189765

In [ ]:
np.mean((np.array(amazon_truth) - np.array(amazon_prediction)) ** 2)

0.4483434046527373

In [ ]:
accuracy(amazon_truth,amazon_prediction)

16.579500198364258 16.701000213623047 16.5684819377262
17.09149932861328 17.159000396728516 16.753355376909898
17.159000396728516 17.000999450683594 17.351826467487285
16.95199966430664 17.118999481201172 16.821307838754315
16.950000762939453 17.297500610351562 16.771902508074774
16.525999069213867 16.559499740600586 16.47545645151391
16.559499740600586 16.194499969482422 16.630787994640826
16.25 16.56599998474121 16.20533226286936
16.181499481201172 16.410499572753906 15.998188196096535
16.096500396728516 16.160499572753906 15.841617382448003
16.090999603271484 16.121999740600586 16.079125987856504
15.873000144958496 15.920499801635742 15.70441061786153
15.848999977111816 16.135000228881836 15.743004509037874
15.322500228881836 15.415499687194824 15.091489304259703
15.415499687194824 15.298500061035156 15.446180687975092
15.182000160217283 15.310500144958496 15.065613912893944
15.648500442504885 15.659000396728516 15.333057656498884
15.659000396728516 14.352999687194824 15.70696879133

0.4989339019189765

In [ ]:
accuracy_map["Amazon"]

[0.5221987315010571, 0.27138410217779574]

predit for next day
sig = each time change difference = average change difference for that model
3 predicction =  predicction + sig , predicction, predicction - sig 
3 obs_confidence = 
final prediction = argmax(predicction * obs_confidence)

In [ ]:
Amazon

,Date,Open,High,Low,Close,Adj Close,Volume,Company
0,2014-08-14,16.389999,16.704500,16.335501,16.660500,16.660500,68408000,Amazon
1,2014-08-15,16.700001,16.750000,16.416000,16.681499,16.681499,77934000,Amazon
2,2014-08-18,16.774000,16.889999,16.688499,16.726500,16.726500,49708000,Amazon
3,2014-08-19,16.743500,16.790501,16.650499,16.756500,16.756500,34276000,Amazon
4,2014-08-20,16.733999,16.858000,16.701000,16.789000,16.789000,36210000,Amazon
...,...,...,...,...,...,...,...,...
469,2016-06-24,34.650002,35.626499,34.610001,34.948002,34.948002,152650000,Amazon
470,2016-06-27,34.600498,34.841000,34.105999,34.568001,34.568001,111360000,Amazon
471,2016-06-28,35.000000,35.400002,34.908501,35.397499,35.397499,80740000,Amazon
472,2016-06-29,35.787498,35.974998,35.676998,35.779999,35.779999,61402000,Amazon


In [ ]:

P =  [0.25, 0.5, 0.25]
O = np.array([ 0.4,0.2,0.4])
P = np.dot(P,O)
P = P/P.sum(axis=0)
